In [1]:
# check which kernel we are using
#!jupyter kernelspec list 

In [1]:
import numpy as np
from numpy.lib.stride_tricks import as_strided
from numba import jit
from skimage.restoration import denoise_nl_means, estimate_sigma
%load_ext line_profiler

In [2]:
def get_dist_loop(s,p1,p2,w,var):
    distance = 0
    for i in range(s):
        for j in range(s):
            tmp_diff = p1[i, j] - p2[i, j]
            distance += w[i, j] * (tmp_diff * tmp_diff - 2 * var)
    distance = max(distance, 0)
    distance = np.exp(-distance)
    return distance


def get_dist_op(s,p1,p2,w,var):
    distance = np.sum(w * ((p1 - p2)**2 - 2 * var))
    distance = max(distance, 0)
    distance = np.exp(-distance)
    return distance

In [5]:
%timeit get_dist_loop(s,a,b,w,0.)
%timeit get_dist_op(s,a,b,w,0.)

147 µs ± 8.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
10.9 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Cython implementation

In [3]:
%load_ext cython

In [4]:
%%cython
import numpy as np
cimport numpy as np
# cdef extern from "fast_exp.h":
#    double fast_exp(double y) nogil
def get_dist_loopx(int s, np.int32_t [:, :] p1, np.int32_t [:, :] p2, np.float64_t [:, :] w, float var):
    cdef double distance = 0., tmp_diff = 0.
    for i in range(s):
        for j in range(s):
            tmp_diff = p1[i, j] - p2[i, j]
            distance += w[i, j] * (tmp_diff * tmp_diff - 2 * var)
    distance = max(distance, 0)
    distance = np.exp(-distance)
    return distance

In [5]:
%%cython
import numpy as np
cimport numpy as np
def get_dist_opx(int s, np.int32_t [:, :] p1, np.int32_t [:, :] p2, np.float64_t [:, :] w, float var):
    cdef double distance = 0.
    distance = np.sum(np.multiply(w,(np.square(np.subtract(p1,p2)) - 2 * var)))
    distance = max(distance, 0)
    distance = np.exp(-distance)
    return distance

In [30]:
%timeit get_dist_loopx(s,a,b,w,0.)
%timeit get_dist_opx(s,a,b,w,0.)
%timeit get_dist_loop(s,a,b,w,0.)
%timeit get_dist_op(s,a,b,w,0.)

3.37 µs ± 78.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
20.5 µs ± 459 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
150 µs ± 2.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
11.3 µs ± 55.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Numba, parallelization

In [6]:
from numba import jit

In [8]:
get_dist_loopn = jit(get_dist_loop)
get_dist_opn = jit(get_dist_op)

In [34]:
%timeit get_dist_loopn(s,a,b,w,0.)
%timeit get_dist_loopx(s,a,b,w,0.)
# %timeit get_dist_op(s,a,b,w,0.)
# %timeit get_dist_loop(s,a,b,w,0.)
timeit get_dist_opn(s,a,b,w,0.)

845 ns ± 34.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
3.36 µs ± 46.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
The slowest run took 14.25 times longer than the fastest. This could mean that an intermediate result is being cached.
3.66 µs ± 5.49 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# matrix optimization (numpy strides)

In [7]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return as_strided(a, shape=shape, strides=strides)


def rolling_block(A, block=(3, 3)):
    shape = (A.shape[0] - block[0] + 1, A.shape[1] - block[1] + 1) + block
    strides = (A.strides[0], A.strides[1]) + A.strides
    return as_strided(A, shape=shape, strides=strides)


def rolling_patch_sets(A, blocks = (3,3), block = (3,3)):
    shape = (A.shape[0] - block[0] + 1 - blocks[0] + 1,
             A.shape[1] - block[1] + 1 - blocks[1] + 1) + blocks + block
    strides = 3*A.strides
    return as_strided(A, shape=shape, strides=strides)

def rolling_apply(fun, a, w):
    r = np.empty(a.shape)
    r.fill(np.nan)
    for i in range(w - 1, a.shape[0]):
        r[i] = fun(a[(i-w+1):i+1])
    return r

In [8]:
def nlmeans_one_loop(a,s,h,var):
    n_channels = a.shape[-1]
    A = ((s - 1.) / 4.)
    offset = (s - 1) / 2
    xg_row, xg_col = np.mgrid[-offset:offset + 1, -offset:offset + 1]
    w = np.ascontiguousarray(np.exp(-(xg_row * xg_row + xg_col * xg_col) / (2 * A * A)),dtype=np.float64)
    w = 1. / (n_channels * np.sum(w) * h * h) * w
    b = np.zeros(a.shape[:-1], dtype='float')
    a_padded = np.pad(a[:,:,0], pad_width=2, mode='reflect')
    rps = rolling_patch_sets(a_padded)
    for index in np.ndindex(a.shape[:-1]):
        weight = np.sum( w * (np.square(rps[index] - rps[index+(1,1)]) - 2. * var), axis = (-1,-2) )
        tmp = -np.maximum(weight, 0)
        weight = np.exp( tmp)
        #weight = ne.evaluate('exp( tmp )')
        weight_sum = np.sum( weight)
        patch_centers = rps[index[0],index[1],:,:,1,1]
        b[index] = np.sum( weight * patch_centers ) / weight_sum
    return b[:,:,None]

In [9]:
%%cython
#cython: initializedcheck=False
#cython: wraparound=False
#cython: boundscheck=False
#cython: cdivision=True
import numpy as np
cimport numpy as np
cimport cython

ctypedef np.float64_t IMGDTYPE

cdef double DISTANCE_CUTOFF = 5.0

cdef inline double patch_dist_2dx(IMGDTYPE [:, :] p1,
                                     IMGDTYPE [:, :] p2,
                                     IMGDTYPE [:, ::] w, int s, double var):
    """
    Compute a Gaussian distance between two image patches.
    Parameters
    ----------
    p1 : 2-D array_like
        First patch.
    p2 : 2-D array_like
        Second patch.
    w : 2-D array_like
        Array of weights for the different pixels of the patches.
    s : int
        Linear size of the patches.
    var : double
        Expected noise variance.
    Returns
    -------
    distance : double
        Gaussian distance between the two patches
    Notes
    -----
    The returned distance is given by
    .. math::  \exp( -w ((p1 - p2)^2 - 2*var))
    """
    cdef int i, j
    cdef int center = (s-1) / 2
    # Check if central pixel is too different in the 2 patches
    cdef double tmp_diff = p1[center, center] - p2[center, center]
    cdef double init = w[center, center] * tmp_diff * tmp_diff
    if init > 1:
        return 0.
    cdef double distance = 0
    for i in range(s):
        # exp of large negative numbers will be 0, so we'd better stop
        if distance > DISTANCE_CUTOFF:
            return 0.
        for j in range(s):
            tmp_diff = p1[i, j] - p2[i, j]
            distance += w[i, j] * (tmp_diff * tmp_diff - 2 * var)
    distance = np.max(distance, 0)
    distance = np.exp(-distance)
    return distance


def nl_means_denoising_2dx(image, int s=7, int d=13, double h=0.1,
                           double var=0.):
    """
    Perform non-local means denoising on 2-D RGB image
    Parameters
    ----------
    image : ndarray
        Input RGB image to be denoised
    s : int, optional
        Size of patches used for denoising
    d : int, optional
        Maximal distance in pixels where to search patches used for denoising
    h : double, optional
        Cut-off distance (in gray levels). The higher h, the more permissive
        one is in accepting patches.
    var : double
        Expected noise variance.  If non-zero, this is used to reduce the
        apparent patch distances by the expected distance due to the noise.
    Notes
    -----
    This function operates on 2D grayscale and multichannel images.  For
    2D grayscale images, the input should be 3D with size 1 along the last
    axis.  The code is compatible with an arbitrary number of channels.
    Returns
    -------
    result : ndarray
        Denoised image, of same shape as input image.
    """
    if s % 2 == 0:
        s += 1  # odd value for symmetric patch
    cdef int n_row, n_col, n_channels
    n_row, n_col, n_channels = image.shape
    cdef int offset = s / 2
    cdef int row, col, i, j, channel
    cdef int row_start, row_end, col_start, col_end
    cdef int row_start_i, row_end_i, col_start_j, col_end_j
    cdef IMGDTYPE [::1] new_values = np.zeros(n_channels).astype(np.float64)
    cdef IMGDTYPE [:, :, ::1] padded = np.ascontiguousarray(np.pad(image,
                       ((offset, offset), (offset, offset), (0, 0)),
                        mode='reflect').astype(np.float64))
    cdef IMGDTYPE [:, :, ::1] result = padded.copy()
    cdef double A = ((s - 1.) / 4.)
    cdef double new_value
    cdef double weight_sum, weight
    xg_row, xg_col = np.mgrid[-offset:offset + 1, -offset:offset + 1]
    cdef IMGDTYPE [:, ::1] w = np.ascontiguousarray(np.exp(
                             -(xg_row * xg_row + xg_col * xg_col) / (2 * A * A)).
                             astype(np.float64))
    cdef double distance
    w = 1. / (n_channels * np.sum(w) * h * h) * w

    # Coordinates of central pixel
    # Iterate over rows, taking padding into account
    for row in range(offset, n_row + offset):
        row_start = row - offset
        row_end = row + offset + 1
        # Iterate over columns, taking padding into account
        for col in range(offset, n_col + offset):
            # Initialize per-channel bins
            for channel in range(n_channels):
                new_values[channel] = 0
            # Reset weights for each local region
            weight_sum = 0
            col_start = col - offset
            col_end = col + offset + 1

            # Iterate over local 2d patch for each pixel
            # First rows
            for i in range(max(-d, offset - row),
                           min(d + 1, n_row + offset - row)):
                row_start_i = row_start + i
                row_end_i = row_end + i
                # Local patch columns
                for j in range(max(-d, offset - col),
                               min(d + 1, n_col + offset - col)):
                    col_start_j = col_start + j
                    col_end_j = col_end + j
                    # Assume grayscale
                    weight = patch_dist_2dx(
                             padded[row_start:row_end,
                                    col_start:col_end, 0],
                             padded[row_start_i:row_end_i,
                                    col_start_j:col_end_j, 0],
                             w, s, var)

                    # Collect results in weight sum
                    weight_sum += weight
                    # Apply to each channel multiplicatively
                    for channel in range(n_channels):
                        new_values[channel] += weight * padded[row + i,
                                                               col + j,
                                                               channel]

            # Normalize the result
            for channel in range(n_channels):
                result[row, col, channel] = new_values[channel] / weight_sum

    # Return cropped result, undoing padding
    return result[offset:-offset, offset:-offset]

In [10]:
@jit
def patch_dist_2dn(p1, p2, w, s, var):
    """
    Compute a Gaussian distance between two image patches.
    Parameters
    ----------
    p1 : 2-D array_like
        First patch.
    p2 : 2-D array_like
        Second patch.
    w : 2-D array_like
        Array of weights for the different pixels of the patches.
    s : int
        Linear size of the patches.
    var : double
        Expected noise variance.
    Returns
    -------
    distance : double
        Gaussian distance between the two patches
    Notes
    -----
    The returned distance is given by
    .. math::  \exp( -w ((p1 - p2)^2 - 2*var))
    """
    center = np.int((s-1) / 2)
    # Check if central pixel is too different in the 2 patches
    tmp_diff = p1[center, center] - p2[center, center]
    init = w[center, center] * tmp_diff * tmp_diff
    if init > 1:
        return 0.
    distance = 0
    for i in range(s):
        # exp of large negative numbers will be 0, so we'd better stop
        if distance > DISTANCE_CUTOFF:
            return 0.
        for j in range(s):
            tmp_diff = p1[i, j] - p2[i, j]
            distance += w[i, j] * (tmp_diff * tmp_diff - 2 * var)
    distance = np.max(distance, 0)
    distance = np.exp(-distance)
    return distance


def nl_means_denoising_2dn(image, s=7, d=13, h=0.1, var=0.):
    """
    Perform non-local means denoising on 2-D RGB image
    Parameters
    ----------
    image : ndarray
        Input RGB image to be denoised
    s : int, optional
        Size of patches used for denoising
    d : int, optional
        Maximal distance in pixels where to search patches used for denoising
    h : double, optional
        Cut-off distance (in gray levels). The higher h, the more permissive
        one is in accepting patches.
    var : double
        Expected noise variance.  If non-zero, this is used to reduce the
        apparent patch distances by the expected distance due to the noise.
    Notes
    -----
    This function operates on 2D grayscale and multichannel images.  For
    2D grayscale images, the input should be 3D with size 1 along the last
    axis.  The code is compatible with an arbitrary number of channels.
    Returns
    -------
    result : ndarray
        Denoised image, of same shape as input image.
    """
    if s % 2 == 0:
        s += 1  # odd value for symmetric patch
    n_row, n_col, n_channels = image.shape
    offset = np.int((s-1) / 2)
    new_values = np.zeros(n_channels).astype(np.float64)
    padded = np.ascontiguousarray(np.pad(image,((offset, offset), (offset, offset), (0, 0)),mode='reflect').astype(np.float64))
    result = padded.copy()
    A = ((s - 1.) / 4.)
    xg_row, xg_col = np.mgrid[-offset:offset + 1, -offset:offset + 1]
    w = np.ascontiguousarray(np.exp(-(xg_row * xg_row + xg_col * xg_col) / (2 * A * A)).astype(np.float64))
    w = 1. / (n_channels * np.sum(w) * h * h) * w

    # Coordinates of central pixel
    # Iterate over rows, taking padding into account
    for row in range(offset, n_row + offset):
        row_start = row - offset
        row_end = row + offset + 1
        # Iterate over columns, taking padding into account
        for col in range(offset, n_col + offset):
            # Initialize per-channel bins
            for channel in range(n_channels):
                new_values[channel] = 0
            # Reset weights for each local region
            weight_sum = 0
            col_start = col - offset
            col_end = col + offset + 1

            # Iterate over local 2d patch for each pixel
            # First rows
            for i in range(max(-d, offset - row),
                           min(d + 1, n_row + offset - row)):
                row_start_i = row_start + i
                row_end_i = row_end + i
                # Local patch columns
                for j in range(max(-d, offset - col),
                               min(d + 1, n_col + offset - col)):
                    col_start_j = col_start + j
                    col_end_j = col_end + j
                    # Assume grayscale
                    weight = patch_dist_2dn(
                             padded[row_start:row_end,
                                    col_start:col_end, 0],
                             padded[row_start_i:row_end_i,
                                    col_start_j:col_end_j, 0],
                             w, s, var)

                    # Collect results in weight sum
                    weight_sum += weight
                    # Apply to each channel multiplicatively
                    for channel in range(n_channels):
                        new_values[channel] += weight * padded[row + i,
                                                               col + j,
                                                               channel]

            # Normalize the result
            for channel in range(n_channels):
                result[row, col, channel] = new_values[channel] / weight_sum

    # Return cropped result, undoing padding
    return result[offset:-offset, offset:-offset]

In [11]:
s = 50
a = np.arange(s**2+s).reshape(s,s+1,1)
b = np.arange(s**2).reshape(s,s)-2
w = 0.5 * np.ones((3,3))
DISTANCE_CUTOFF = 5.0

In [15]:
%%timeit 
r0 = nl_means_denoising_2dx(a,s=3, d=2, h=0.1)

18.4 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit 
r1 = denoise_nl_means(a,h=0.1,fast_mode=True,patch_size=3,patch_distance=2,multichannel=False)

22 ms ± 2.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
s,h = 3,.1
r2 = nlmeans_one_loop(a,s=3,h=0.1,var=0)

81.4 ms ± 3.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit 
r3 = nl_means_denoising_2dn(a,s=3, d=2, h=0.1)

331 ms ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
np.allclose(r0,r1)

True

In [49]:
# %prun -l 5 nl_means_denoising_2dn(a,s=3, d=2, h=0.1)
%prun -l 5 nlmeans_one_loop(a,s=3,h=0.1,var=0)